<a href="https://colab.research.google.com/github/ras1364/Fake-Review-Using-BERT/blob/main/FakeReviewDetectionUsingBertVer3_1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**In this code, comments are sent to the BERT encoder regardless of the number of sentences, and its output is sent to the SVM classifier.**

In [1]:
import pandas as pd
import numpy as np

In [2]:
# from google.colab import files
# uploaded = files.upload()

In [2]:
X_test = pd.read_csv("/content/X_test.csv")
X_train = pd.read_csv("/content/X_train.csv")
y_test = pd.read_csv("/content/y_test.csv")
y_train = pd.read_csv("/content/y_train.csv")

In [3]:
max_length = 100
# for x in range(0,len(X_train)):
#   line = X_train.loc[x].at["text"]
#   if max_length < len(line.split()) :
#     max_length = len(line.split())

print(max_length)


100


In [4]:
from gensim.parsing.preprocessing import remove_stopwords

def preprocess(text):
  return remove_stopwords(text)

X_train["stemmed_text_data"] = X_train["text"].map(preprocess)
X_test["stemmed_text_data"] = X_test["text"].map(preprocess)

In [ ]:
# !pip install transformers

In [5]:
from transformers import AutoTokenizer, TFBertModel 
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [34]:
# For Test
# encoded = tokenizer.encode_plus("Hello beautifull human, i'm Reza", return_tensors="tf")
# embedding = bert_model(encoded['input_ids'],encoded['attention_mask'])

In [70]:
def encode(text, maxlen):
  encoded = tokenizer.encode_plus(text,max_length=maxlen, return_tensors="tf")
  embedding = bert_model(encoded['input_ids'],encoded['attention_mask'])[1]
  return embedding

In [75]:
X_train_embedding = []
X_test_embedding = []
for i in range(len(X_train.stemmed_text_data)):
  X_train_embedding.append(encode(X_train.stemmed_text_data[i], maxlen=max_length))
for i in range(len(X_test.stemmed_text_data)):
  X_test_embedding.append(encode(X_test.stemmed_text_data[i],maxlen=max_length))


In [78]:
print("X_train.shape = " , np.shape(X_train_embedding), " y_train.shape = ", y_train.deceptive.shape)
print("X_test.shape = " , np.shape(X_test_embedding), " y_test.shape = ", y_test.deceptive.shape)

X_train.shape =  (1200, 1, 768)  y_train.shape =  (1200,)
X_test.shape =  (400, 1, 768)  y_test.shape =  (400,)


In [81]:
X_train_embedding_squeezed = np.squeeze(X_train_embedding)
X_test_embedding_squeezed = np.squeeze(X_test_embedding)

In [92]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X_train_embedding_squeezed)
X_train_std = sc.transform(X_train_embedding_squeezed)
X_test_std = sc.transform(X_test_embedding_squeezed)

In [93]:
np.shape(X_train_std)

(1200, 768)

In [94]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train_std, y_train.deceptive)

SVC(random_state=0)

In [95]:
y_pred = classifier.predict(X_test_std)

In [98]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test.deceptive, y_pred)
print(cm)
print("Accuracy : ",accuracy_score(y_test.deceptive,y_pred))

[[105  95]
 [ 79 121]]
Accuracy :  0.565
